In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pandas as pd

/tmp/ipykernel_332731/362687639.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
save_dir = '/home/jl277509/Documents/Ablation_translation_3/trimdepth.csv'

# Step 1 : recover all perf values

In [ ]:
# instead of printing, add in table
from pathlib import Path
import pandas as pd

basedir='/neurospin/dico/jlaval/Output/8_trimdepth_translation_3'

sub_dirs = {'SOr': [f'{basedir}/SOr_left_UKB40/', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'test_weighted_auc'],
            'FIP': [f'{basedir}/FIP_right_UKB40/', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'test_auc'],
            'SC-sylv': [f'{basedir}/SC-sylv_left_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_central_left_dim{k}/test_values.json' for k in range(1,7)], 'test_r2']}

l = os.listdir(basedir)
root_names = [elem for elem in l if 'UKB40' not in elem]

In [10]:
# store everything globally
l_model_glob = []
l_perf_glob = []
l_region_glob = []
l_task_glob = []


for region, (sub_dir, auc_files, metric) in sub_dirs.items(): 
    for auc_file in auc_files:
        models = os.listdir(sub_dir)
        models = [elem for elem in models if 'config' not in elem and 'multirun' not in elem]
        models.sort()
        task = Path(auc_file).parts[-2]
        for name in root_names:
            #l_models = [elem for elem in models if elem.startswith(name)]
            l_models = [elem for elem in models if name in elem]
            l_perfs = []
            for model in l_models:
                try:
                    with open(os.path.join(sub_dir, model, auc_file), 'r') as f:
                        perf = json.load(f)
                        if isinstance(perf[metric], float):
                            value = perf[metric]
                        else:
                            value = perf[metric][0]
                        l_perfs.append(value)
                except Exception:
                    print(f'{os.path.join(sub_dir, model, auc_file)} not found') ## NB : can't continue if not all files found ...
            l_model_glob.append(l_models)
            l_perf_glob.append(l_perfs)
            l_region_glob.append([region for _ in range(len(l_perfs))])
            l_task_glob.append([task for _ in range(len(l_perfs))])

In [23]:
# reformat lists and put inside DataFrame
def flatten_sum(matrix):
    return sum(matrix, [])
cols = ['model', 'region', 'task', 'score']
values = np.array([flatten_sum(l_model_glob), flatten_sum(l_region_glob), flatten_sum(l_task_glob), flatten_sum(l_perf_glob)]).T
df = pd.DataFrame(data=values, columns=cols)

In [24]:
df

,model,region,task,score
0,10_trimdepth_6mm_constant_p40name13-15-21_96,SOr,Left_OFC,0.4475222843901367
1,5_trimdepth_4mm_constant_p20name15-08-32_130,SOr,Left_OFC,0.549519400414767
2,8_trimdepth_4mm_per_branch_p40name16-05-05_121,SOr,Left_OFC,0.49164404683796226
3,3_trimdepth_2mm_per_branch_p20name14-34-11_53,SOr,Left_OFC,0.5303061991653146
4,2_trimdepth_2mm_constant_p40name14-26-52_225,SOr,Left_OFC,0.5642814032120501
...,...,...,...,...
91,7_trimdepth_4mm_per_branch_p20name15-39-56_223,SC-sylv,Isomap_central_left_dim6,0.03614785884445493
92,12_trimdepth_6mm_per_branch_p40name13-53-31_106,SC-sylv,Isomap_central_left_dim6,-0.004040946673093959
93,4_trimdepth_2mm_per_branch_p40name14-50-16_244,SC-sylv,Isomap_central_left_dim6,0.031300190465400224
94,6_trimdepth_4mm_constant_p40name15-28-57_242,SC-sylv,Isomap_central_left_dim6,0.048498503358817


# Step 2 : reorganize the df with the parameters

In [25]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_mode=[]
list_proba=[]
for model in df['model'].tolist():
    if '2mm' in model:
        list_magnitude.append(2)
    elif '4mm' in model:
        list_magnitude.append(4)
    elif '6mm' in model:
        list_magnitude.append(6)

    if 'constant' in model:
        list_mode.append('constant')
    else:
        list_mode.append('per branch')

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

In [26]:
df['Magnitude']=list_magnitude
df['Mode']=list_mode
df['Proba'] = list_proba

In [27]:
df_reformat = df.copy()
df_reformat = df_reformat.drop(labels='model', axis='columns')


In [28]:
# Pivoting the table
df_pivot = df_reformat.pivot(index=['Magnitude', 'Mode', 'Proba'], columns=['task', 'region'], values='score')

# Renaming columns
df_pivot.columns = [f'score_{col}' for col in df_pivot.columns]

# Resetting index to get a flat DataFrame
df_pivot = df_pivot.reset_index()

df_pivot['score_Isomap_left_global'] = df_pivot[df_pivot.columns[-6:]].astype(float).mean(axis=1)
cols = df_pivot.columns.tolist()
cols = cols[:5]+cols[-1:]+cols[5:-1] # put average Isomap before
df_pivot=df_pivot[cols] # swap cols

In [29]:
# *100 and keep only 1 decimal place
score_cols = df_pivot.columns[3:].tolist()
df_pivot[score_cols]=df_pivot[score_cols].apply(pd.to_numeric).mul(100).round(1)

In [30]:
df_pivot

,Magnitude,Mode,Proba,"score_('Left_OFC', 'SOr')","score_('Right_FIP', 'FIP')",score_Isomap_left_global,"score_('Isomap_central_left_dim1', 'SC-sylv')","score_('Isomap_central_left_dim2', 'SC-sylv')","score_('Isomap_central_left_dim3', 'SC-sylv')","score_('Isomap_central_left_dim4', 'SC-sylv')","score_('Isomap_central_left_dim5', 'SC-sylv')","score_('Isomap_central_left_dim6', 'SC-sylv')"
0,2,constant,20,66.2,57.9,12.5,11.9,12.7,21.4,17.5,9.3,2.2
1,2,constant,40,56.4,64.0,17.1,15.2,18.6,24.8,23.1,16.3,4.5
2,2,per branch,20,53.0,50.7,13.3,12.5,14.4,17.9,19.1,11.4,4.5
3,2,per branch,40,53.0,59.3,15.8,15.7,18.0,24.2,20.8,13.2,3.1
4,4,constant,20,55.0,56.1,16.7,17.8,18.9,27.5,18.6,11.7,5.8
5,4,constant,40,53.0,62.9,17.2,18.7,19.0,29.5,21.7,9.6,4.8
6,4,per branch,20,54.9,63.3,13.8,16.2,16.1,19.3,18.3,9.0,3.6
7,4,per branch,40,49.2,60.1,16.7,20.3,21.1,22.5,23.7,8.5,4.0
8,6,constant,20,51.4,53.8,29.5,38.1,31.4,39.2,36.7,19.2,12.5
9,6,constant,40,44.8,58.6,17.5,17.1,21.4,29.8,24.1,9.3,3.1


In [22]:
df_pivot.to_csv(save_dir, index=False)